In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 61.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import classification_report
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW# get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import time

!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/utils.py .
from utils import *
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/Custom_Dataset_Class.py .
from Custom_Dataset_Class import CustomDataset
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/pytorchtools.py .
from pytorchtools import EarlyStopping
#from Bert_Classification import Bert_Classification_Model
#from RoBERT import RoBERT_Model

#from BERT_Hierarchical import BERT_Hierarchical_Model
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [4]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [6]:
%matplotlib inline

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [7]:
#change to where you store mimic3 data
MIMIC_3_DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/datasets'

train_df = pd.read_csv('%s/train_50.csv' % MIMIC_3_DIR)
eval_df = pd.read_csv('%s/dev_50.csv' % MIMIC_3_DIR)
test_df = pd.read_csv('%s/test_50.csv' % MIMIC_3_DIR)

train_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,23657,125544,Physician,title,584.9;403.91;311,1
1,30845,115327,Nursing/other,addendum,276.0;584.9;287.5;518.81;428.0,1
2,81475,101662,Physician,title,250.00;785.52;427.31;995.92;530.81;584.5;038.9...,1
3,70220,163685,Nursing,dnr,511.9;276.0;584.9;995.92;414.01;285.1;038.9;51...,1
4,5331,142049,Nursing/other,hypotension,250.00;401.9;427.31;530.81;414.01,1


In [8]:
full_df = pd.concat([train_df, eval_df, test_df], ignore_index=True)

In [9]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,23657,125544,Physician,title,"[584.9, 403.91, 311]",1
1,30845,115327,Nursing/other,addendum,"[276.0, 584.9, 287.5, 518.81, 428.0]",1
2,81475,101662,Physician,title,"[250.00, 785.52, 427.31, 995.92, 530.81, 584.5...",1
3,70220,163685,Nursing,dnr,"[511.9, 276.0, 584.9, 995.92, 414.01, 285.1, 0...",1
4,5331,142049,Nursing/other,hypotension,"[250.00, 401.9, 427.31, 530.81, 414.01]",1


In [10]:
#load multi label binarizer for one-hot encoding
mlb = MultiLabelBinarizer(sparse_output=True)

#labels_onehot = mlb.fit_transform(train_df.pop('LABELS'))
#labels_onehot[0][1]

In [11]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61
0,23657,125544,Physician,title,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30845,115327,Nursing/other,addendum,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,81475,101662,Physician,title,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,70220,163685,Nursing,dnr,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,5331,142049,Nursing/other,hypotension,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373477,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373478,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373479,91939,149837,Discharge summary,admission date discharge date date of birth se...,5630,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373480,96777,176399,Discharge summary,admission date discharge date date of birth se...,5890,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
## Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

#full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()


,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61
0,23657,125544,Physician,title,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30845,115327,Nursing/other,addendum,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,81475,101662,Physician,title,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,70220,163685,Nursing,dnr,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,5331,142049,Nursing/other,hypotension,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
full_df = full_df.drop(full_df[full_df['length']<300].index)

In [14]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61
220400,605,115545,Nursing,pt is an yo woman with uti complicated by sept...,300,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
220401,1766,185657,Nursing/other,t sicu npn review of systems neuro alert orien...,300,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
220402,29348,127366,Nursing/other,condition update see carevue for specifics rem...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
220403,18447,159812,Nursing/other,tsicu npn o ros neuro sedated on propofol w pr...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
220404,2468,133778,Nursing/other,npn bili resp pt remains on simv with settings...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373477,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373478,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373479,91939,149837,Discharge summary,admission date discharge date date of birth se...,5630,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373480,96777,176399,Discharge summary,admission date discharge date date of birth se...,5890,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [15]:
train_df, test_df = train_test_split(full_df, test_size=0.2)

In [16]:
train_df, dev_df = train_test_split(train_df, test_size=0.2)

In [17]:
train_df.sort_values(['length'], inplace=True)
dev_df.sort_values(['length'], inplace=True)
test_df.sort_values(['length'], inplace=True)


In [18]:
train_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61
0,76975,141118,Nursing,cardiac arrest assessment neuro exam as below ...,300,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,93640,175706,Nutrition,subjective patient intubated and sedated objec...,300,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19199,103143,Nursing/other,npn o infant continues on conventional vent se...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0
3,2984,155981,Nursing,this is a year old female pt with h o breast c...,300,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,98634,131717,Echo,patient test information indication left ventr...,300,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19127,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19128,86359,164038,Discharge summary,admission date discharge date date of birth se...,5456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
19129,19569,197451,Discharge summary,admission date discharge date date of birth se...,5747,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19130,95323,142423,Discharge summary,admission date discharge date date of birth se...,6116,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
test_pred = test_df.copy()

In [20]:
X_train = train_df.TEXT
X_test = test_df.TEXT


NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in icd_classes_50:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train_df[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    test_pred[category] = prediction


... Processing 008.45
... Processing 038.9
... Processing 244.9
... Processing 250.00
... Processing 272.0
... Processing 272.4
... Processing 276.0
... Processing 276.1
... Processing 276.2
... Processing 285.1
... Processing 285.9
... Processing 287.5
... Processing 305.1
... Processing 311
... Processing 401.9
... Processing 403.90
... Processing 403.91
... Processing 410.71
... Processing 412
... Processing 414.01
... Processing 427.31
... Processing 428.0
... Processing 486
... Processing 496
... Processing 507.0
... Processing 511.9
... Processing 518.0
... Processing 518.5
... Processing 518.81
... Processing 530.81
... Processing 584.5
... Processing 584.9
... Processing 585.9
... Processing 599.0
... Processing 747.0
... Processing 769
... Processing 770.81
... Processing 774.2
... Processing 776.6
... Processing 779.3
... Processing 779.81
... Processing 785.52
... Processing 995.92
... Processing V05.3
... Processing V29.0
... Processing V30.00
... Processing V30.01
... Proc

In [21]:
test_df['labels'] = test_df[icd_classes_50].values.tolist()
test_pred['labels'] = test_pred[icd_classes_50].values.tolist()

In [22]:
outputs = np.vstack([test_pred['labels'][i] for i in test_pred.index])
targets = np.vstack([test_df['labels'][i] for i in test_df.index])

In [23]:
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.16316923915467135
F1 Score (Macro) = 0.04702202057480678


In [24]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.0000    0.0000    0.0000       969
       038.9     0.8125    0.0044    0.0087      2971
       244.9     0.3333    0.0004    0.0008      2469
      250.00     1.0000    0.0022    0.0043      3674
       272.0     0.3333    0.0006    0.0012      1682
       272.4     0.6667    0.0016    0.0032      3770
       276.0     0.0000    0.0000    0.0000      2337
       276.1     0.3333    0.0005    0.0010      1973
       276.2     0.6667    0.0013    0.0025      3168
       285.1     0.0000    0.0000    0.0000      2452
       285.9     0.0000    0.0000    0.0000      2701
       287.5     0.0000    0.0000    0.0000      2107
       305.1     0.0000    0.0000    0.0000      1521
         311     0.0000    0.0000    0.0000      1640
       401.9     0.6439    0.2324    0.3416      7163
      403.90     0.0000    0.0000    0.0000      2210
      403.91     0.3333    0.0009    0.0018      1090
      410.71     0.0000    

In [25]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.5437813817991677
RUC AUC Score (Macro) = 0.5138077558776164
